# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [ ]:
!python -c "import platform; print(platform.platform()); import sys; print(sys.version)"

In [1]:
from engine import SearchEngine
from evaluate_map import *
from optimizing import Optimize

C:\Users\ariel\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
se = SearchEngine()

In [4]:
se.set_searcher(approach="qld")

In [ ]:
results = se.get_top_k("international organized crime", k=50)

In [ ]:
results[0][0][:300]

In [2]:
topics = load_topics("Data/queriesROBUST.txt")

In [6]:
# This one creates a file called "run.txt" with submitting format, can change file name
se.search_all_queries(topics, 100)

In [7]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("run.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

0.24594939993995996

In [7]:
map = get_map_by_paths("Data/qrels_50_Queries", "run.txt")

In [8]:
map

0.21283191880598026

In [3]:
opti = Optimize()
# fb_terms_values = [5, 6, 8, 10, 15, 20]
# fb_docs_values = [5, 7, 10, 15]
# og_query_weight_values = [0.3, 0.4, 0.5, 0.6, 0.7]
mus = [200,300,400,500,600,700,800,900,1000,1100,1200]

opti.optimize_qld(topics, [20], [5], [0.6], mus, k=1000)

fb_terms=20, fb_docs=5, w=0.6, mu=200 -> MAP=0.259288
fb_terms=20, fb_docs=5, w=0.6, mu=300 -> MAP=0.264891
fb_terms=20, fb_docs=5, w=0.6, mu=400 -> MAP=0.264188
fb_terms=20, fb_docs=5, w=0.6, mu=500 -> MAP=0.263658
fb_terms=20, fb_docs=5, w=0.6, mu=600 -> MAP=0.262625
fb_terms=20, fb_docs=5, w=0.6, mu=700 -> MAP=0.259600
fb_terms=20, fb_docs=5, w=0.6, mu=800 -> MAP=0.256631
fb_terms=20, fb_docs=5, w=0.6, mu=900 -> MAP=0.254628
fb_terms=20, fb_docs=5, w=0.6, mu=1000 -> MAP=0.253824
fb_terms=20, fb_docs=5, w=0.6, mu=1100 -> MAP=0.251106
fb_terms=20, fb_docs=5, w=0.6, mu=1200 -> MAP=0.249275

BEST:
fb_terms=20, fb_docs=5, w=0.6, mu=300 -> MAP=0.264891
